## Project 1
The goal for project 1 is to create a controller to land a rocket. In order to do this, I am utilizing Pytorch's neural network and machine learning algorithms. This learning algorithm will use the dynamics of the rocket to plot the trajectory and overall state of the rocket. It will then modifiy the action variables related to thrust and angle of thrust to reach the optimum final state. This optimization uses a loss function that incorporates both the final state and the transient states of the rocket. Through many iterations, a neural network controller is developed so the rocket can land safely on the ground.

### Dynamics
![dynamics](\dynamics.png)

The rocket has two forces acting on it. the force due to gravity at the center of mass and the force due to the thruster at the bottom of the rocket. For this project, the dynamics are 2D rigid body dynamics for a simpler controller. The following are the equations of motion
\begin{equation*}
\begin{aligned}
F_{thruster} \mathrm{sin} (\phi + \theta) = m_{rocket} a_x \\
F_{thruster} \mathrm{cos} (\phi + \theta) - F_g = m_{rocket} a_y \\
F_{thruster} L_{cg} \mathrm{sin}(\phi) = I_{cg} \alpha
\end{aligned}
\end{equation*}
Incase the photo of the dynamics did not load, $ \theta \ $ is the angle between the central axis of the rocket and the y axis. $\phi \ $ is the angle between the rockets central axis and the angle of the thruster.
<br>
These three equations can be put into terms of purely accelerations which is used for the learning algorithym
\begin{equation*}
\begin{aligned}
a_x = \frac {F_{th}}{m_r} \mathrm{sin}(\phi + \theta) \\
a_y = \frac {F_{th}}{m_r} \mathrm{cos}(\phi + \theta) - g \\
\alpha = \frac {F_{th} L_{cg}}{I_{cg}} \mathrm{sin}(\phi) = \frac {4 F_{th}}{L_{cg} m_r} \mathrm{sin}(\phi)
\end{aligned}
\end{equation*}

Here $ \frac {F_{th}}{m_r}\ $ can be assigned a full thrust acceleration value that is greater than gravity g. For this purpose, the thrust was chosen to be 14.715 m/s^2. These equations can be descritized for solving purposes. The matrix of these equations is as follows
\begin{equation}
\begin{bmatrix}
y(t+1) \\ \dot{y}(t+1) \\ x(t+1) \\ \dot{x}(t+1) \\ \theta(t+1) \\ \omega(t+1)
\end{bmatrix}
 = \begin{bmatrix}
 y(t) + \dot{y}(t)\delta t \\ \dot{y} + a_y(t)\delta t \\ x(t) + \dot{x}(t)\delta t \\
 \dot{x}(t) + a_x(t)\delta t \\ \theta (t) + \omega (t)\delta t \\ \omega(t) + \alpha (t)\delta t
 \end{bmatrix}
 \end{equation}
 To drive these dynamics, there are two aspects that the controller controls. Thrust and $\phi\ $. The thrust variable will vary from 0 to 1 and multiply by the thrust acceleration. the $\phi\ $ angle can vary from -0.5 radians to 0.5 radians; this is about a 29 degree tilt in each direction. <br><br>
 For the formulation of this problem, all the position variables (not $\phi\ $ and $\theta$) are normalized by the average starting height of 1000 meters. The time step is set for 1.5 seconds and 25 steps are taken. These settings lead to the most stable simulations and convergence. These equations are extendable to multiple initial conditions by doing batch multiplications to compute sequential states for all initial states at the same time. For this problem, 5 randomly generated initial states with normal distributions were created.<br>
 $y\ $ has an average of 1 with a standard diviation of 0.05<br>
 $\dot{y}\ $ is randomly generated between [0, -0.005] <br>
 $x\ $ has an average of 0 with a  standard diviation of 0.05 <br>
 $\dot{x}\ $ has an average of 0 with a standard deviation of 0.005 <br>
 $\theta\ $ has an average of 0 with a standard deviation of 0.1 radians <br>
 $\omega\ $ has an average of 0 with a standard deviation of 0.05 radians/s <br>

### Optimization and Loss Function
The neural network is optimizing the actions taken by the rocket. The network utlized a loss funtion that is differentiable to approach an optimum solution. This loss function has two parts, the terminal loss and the transitional loss.<br>
The terminal loss is as follows with T being the final timestep:

\begin{equation}
\mathrm{terminal\ error} = 10 * (y(T) - L_{cg})^2 + 10 * (\dot{y} (T))^2 + x(T)^2 + 2 * \dot{x} (T)^2 + \theta (T)^2 + \omega (T)^2
\end{equation}
The transitional error was used to make the network converge faster and maintain stability for convergence. The components are as follows:
\begin{equation*}
\begin{aligned}
& x_{error} = \sum_{i = 1}^{T-1}(x(t_i)^2) \\
& \theta_{error} = \sum_{i =1}^{T-1}(\theta(t_i)^2 \\
& \dot{y} _{error} = \sum_{i = T-4}^{T-1}(\dot{y} (t_i)^2) \\
& y_{error} = \sum_{i = 1}^{T-1}(y(t_i) - L_{cg})^2 & & \forall \ y(t_i) < 0 \\
& \omega_{error} = \sum_{i = T-6}^{T-1} \omega (t_i)^2
\end{aligned}
\end{equation*}
\begin{equation}
\mathrm{transition\ error} = 0.03 * x_{error} + 0.07 * \theta _{error} + 0.8 * \dot{y} _{error} + 0.06 * \omega _{error} + 0.8 * {y_error}
\end{equation}

$\mathrm{loss} = \mathrm{terminal\ error} + \mathrm{transition\ error}$



In [ ]:
import logging
import math
import random
import numpy as np
import time

import torch
import torch as t
import torch.nn as nn
from torch import optim
from torch.nn import utils
import matplotlib.pyplot as plt

t.manual_seed(13)

logger = logging.getLogger(__name__)

FRAME_TIME = 1.5
GRAVITY_ACCEL = 9.81 / 1000
BOOST_ACCEL = 14.715 / 1000
L_center_of_gravity = 5 / 1000

PLATFORM_WIDTH = 25 / 1000


class Dynmaics(nn.Module):

    def __init__(self):
        super(Dynmaics, self).__init__()

    @staticmethod
    def forward(state, action):
        """
        action[0]: thrust
        action[1]: phi of thrust
        state[0] = y
        state[1] = y_dot
        state[2] = x
        state[3] = x_dot
        state[4] = theta
        state[5] = omega
        """
        delta_state_gravity = t.tensor([0., -GRAVITY_ACCEL * FRAME_TIME, 0., 0., 0., 0])
        N = len(state)
        # Thrust Calculations
        state_tensor = torch.zeros((N, 6))
        state_tensor[:, 1] = torch.cos(state[:, 4] + action[:, 1])
        state_tensor[:, 3] = torch.sin(state[:, 4] + action[:, 1])
        delta_state_thrust = BOOST_ACCEL * FRAME_TIME * t.mul(state_tensor, action[:, 0].reshape(-1, 1))

        state_tensor_angle = t.zeros((N, 6))
        state_tensor_angle[:, 5] = 4 / L_center_of_gravity * t.sin(action[:, 1]) * action[:, 0]
        delta_state_angle = BOOST_ACCEL * FRAME_TIME * state_tensor_angle

        state = state + delta_state_thrust + delta_state_gravity + delta_state_angle

        step_mat = t.tensor([[1., FRAME_TIME, 0., 0., 0., 0.],
                             [0., 1., 0., 0., 0., 0.],
                             [0., 0., 1., FRAME_TIME, 0., 0.],
                             [0., 0., 0., 1., 0., 0.],
                             [0., 0., 0., 0., 1., FRAME_TIME],
                             [0., 0., 0., 0., 0., 1]])
        state = t.t(t.matmul(step_mat, state.t()))

        return state


class Controller(nn.Module):

    def __init__(self, dim_input, dim_hidden, dim_output):
        """
        dim_input: # of system states
        dim_output: # of actions
        dim_hidden: up to me
        """

        super(Controller, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(dim_input, dim_hidden),
            nn.Tanh(),
            nn.Linear(dim_hidden, dim_output),
            nn.Sigmoid(),
            nn.Linear(dim_output, dim_input),
            nn.Tanh(),
            nn.Linear(dim_input, dim_output),
            nn.Tanh()
        )

    def forward(self, state):
        action = self.network(state)
        action = action / 2 + t.tensor([0.5, 0.])
        return action


class Simulation(nn.Module):

    def __init__(self, controller, dynamics, T):
        super(Simulation, self).__init__()
        self.state = self.initialize_state()
        self.controller = controller
        self.dynamics = dynamics
        self.T = T
        self.action_trajectory = []
        self.state_trajectory = []

    def forward(self, state):
        self.action_trajectory = []
        self.state_trajectory = []
        for _ in range(T):
            action = self.controller.forward(state)
            state = self.dynamics.forward(state, action)
            self.action_trajectory.append(action)
            self.state_trajectory.append(state)
        return self.error(state, self.state_trajectory)

    @staticmethod
    def initialize_state():
        y = np.random.normal(1.0, 0.05, 5)
        y_dot = np.random.rand(5) * -0.005
        x = np.random.normal(0., 0.05, 5)
        x_dot = np.random.normal(0., 0.005, 5)
        theta = np.random.normal(0., 0.1, 5)
        omega = np.random.normal(0., 0.05, 5)
        state = np.zeros((len(y), 6))
        for i in range(len(y)):
            state[i, :] = [y[i], y_dot[i], x[i], x_dot[i], theta[i], omega[i]]# need initial conditions
        return t.tensor(state, requires_grad=False).float()

    def error(self, state, state_trajectory):
        termination_error = 10 * (state[:, 0] - L_center_of_gravity) ** 2 + 10 * state[:, 1] ** 2 + state[:, 2] ** 2 + 2 * state[:, 3] ** 2 + state[:, 4] ** 2 + state[:, 5] ** 2
        termination_error = t.sum(termination_error)
        stack_state_traj = t.stack(state_trajectory)
        x_location = stack_state_traj[:-1, :, 2]
        x_location_error_squared = t.sum(x_location**2, (1, 0))
        angle = stack_state_traj[:-1, :, 4]
        angle_error_squared = t.sum(angle**2, (0, 1))
        y_speed = stack_state_traj[-4:-1, :, 1]
        y_speed_error_squared = t.sum(y_speed**2, (1, 0))
        y = stack_state_traj[:-1, :, 0]
        y = t.nn.functional.relu(-(y - L_center_of_gravity))
        y_squared_error = t.sum(y**2, (1, 0))
        omega = stack_state_traj[-6:-1, :, 5]
        omega_error_sq = t.sum(omega**2, (1, 0))
        transition_error = 0.03 * x_location_error_squared + 0.07 * angle_error_squared + 0.8 * y_speed_error_squared + 0.06 * omega_error_sq  + 0.8 * y_squared_error

        return termination_error + transition_error


class Optimize:
    def __init__(self, simulation):
        self.simulation = simulation
        self.parameters = simulation.controller.parameters()
        self.optimizer = optim.LBFGS(self.parameters, lr=0.012)

    def step(self):
        def closure():
            loss = self.simulation(self.simulation.state)
            self.optimizer.zero_grad()
            loss.backward()
            return loss

        self.optimizer.step(closure)
        return closure()

    def train(self, epochs):
        for epoch in range(epochs):
            loss = self.step()
            print('[%d] loss: %.5f' % (epoch + 1, loss))
            self.visualize()


    def visualize(self):
        data = np.array([self.simulation.state_trajectory[i].detach().numpy() for i in range(self.simulation.T)])
        N_cond = len(data[0, :, 0])
        fig = plt.figure(tight_layout=True)
        import matplotlib.gridspec as grsp
        gs = grsp.GridSpec(3, 2)
        axyydot = fig.add_subplot(gs[0, :])
        axx = fig.add_subplot(gs[1, 0])
        axxdot = fig.add_subplot(gs[1, 1])
        axthet = fig.add_subplot(gs[2, 0])
        axomeg = fig.add_subplot(gs[2, 1])
        for i in range(N_cond):
            y = data[:, i, 0]
            y_dot = data[:, i, 1]
            x = data[:, i, 2]
            x_dot = data[:, i, 3]
            theta = data[:, i, 4]
            omega = data[:, i, 5]

            axyydot.plot(y, y_dot)
            axx.plot(y, x)
            axxdot.plot(y, x_dot)
            axthet.plot(y, theta)
            axomeg.plot(y, omega)

        axyydot.set_xlabel('Normalized Vertical Distance')
        axx.set_xlabel('Normalized Vertical Distance')
        axxdot.set_xlabel('Normalized Vertical Distance')
        axthet.set_xlabel('Normalized Vertical Distance')
        axomeg.set_xlabel('Normalized Vertical Distance')
        axyydot.set_ylabel('Normalized y_dot')
        axx.set_ylabel('Normalized x')
        axxdot.set_ylabel('Normalized x_dot')
        axthet.set_ylabel('Theta of Rocket')
        axomeg.set_ylabel('Omega of Rocket')
        fig.align_labels()
        plt.show()


T = 25
dim_input = 6
dim_hidden = 12
dim_output = 2
d = Dynmaics()
c = Controller(dim_input, dim_hidden, dim_output)
s = Simulation(c, d, T)
o = Optimize(s)
o.train(230)
